# Getting song information from spotify

In [1]:
import pandas as pd
import time
import sys
sys.path.append('../../Source')
from source import *

There is already a list of songs from a previous notebook, so we use that to search Spotify.

In [2]:
df_songs = pd.read_fwf('../../Data/Raw/unique_songs.txt', header=None, names=['SongAndArtist'])
print(df_songs.describe())

                                            SongAndArtist
count                                              204470
unique                                             204428
top     Nightwish - All the Works of Nature Which Ador...
freq                                                    8


For every song name we have, we search spotify and get the top result. That might lead to duplicates, but they should be rare enough to not matter and we can easily filter them out using the spotify id.
<br>
We then save them into Data/Raw/unique_songs.txt

20 hours estimate for all 200k songs - first 1k in 6 mins; first 10k in 58mins
<br>
However, Spotify API limits this to 25k songs a day, in about 150 minutes.

In [ ]:
access_token = get_token()
with open('../../Data/Raw/spotify_song_data_3.txt', "a+", encoding="utf-8") as f:
    for index, row in df_songs.iloc[50000:75000].iterrows():
        this_song = (index, row['SongAndArtist'])
        if index % 1000 == 0:
            print("Index: ", index)
        # The token expires in around an hour, so we need to get a new one every so often
        if index % 8000 == 0:
            access_token = get_token()
        try:
            track_info = get_tracks(access_token, row['SongAndArtist'], limit=1, offset=0)
            # print(track_info.json()['tracks']['items'])
            f.write(str(track_info.json()['tracks']["items"][0]) + '\n')
        except Exception as e:
            print("Failed to get a track: ", this_song)
            try:
                print("With Python throwing: ", e)
                print("With the response message: ", track_info.json())
            except Exception as e:
                # We don't get the tempalted response from spotify, meaning we get a error message instead
                # Very likely just a 429 error, so we wait a bit and try again
                # Or in some cases other issues, leading to 529/other errors
                print(track_info)
                time.sleep(300)
            access_token = get_token()

        time.sleep(0.1)

Access token: BQC5xglCAwBTDzEB4v7sFbONUM_sIOEMUZ14aqMyezHGKkzELvExBW2-Ox3JEFCzSo6JEbPDixvQ9GsWjDUEJ_nafighg_Pm1FPrVlIvr5AwlNLeVGr_WY7ITROuJWcb9iRAT5ajE3I
Index:  50000
Failed to get a track:  (50411, 'W Sound - Mi Novio Tiene Novia - W Sound 02')
With Python throwing:  Expecting value: line 1 column 1 (char 0)
<Response [504]>
Access token: BQDISlpIwKPqwW-M_KgvL7ti_EwGB3HpP1WDIpj1a_3UnQ3GXYjPDvx07K8gHSXR8Yti3hF7yeXocjI2eLwPP-6TUSrj8g2W7P8Q5V2Hf6TdrxY4lQx4_QRZZ-tEoLVB3ozJbw2pUgk
Failed to get a track:  (50619, 'Ricardo Ray - Bella Es la Navidad')
With Python throwing:  ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
With the response message:  {'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=Los%20Graduados%20-%20La%20Pelea%20Del%20Siglo&type=track', 'limit': 1, 'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=Los%20Graduados%20-%20La%20Pelea%20Del%20Siglo&type=track', 'offset': 0, 'previous': Non

In [2]:
deezer = pd.read_csv('../../Data/Processed/deezer_tracks_super_dataset1_final.csv')

In [3]:
deezer = deezer[deezer['rank'] != 0]
deezer.drop_duplicates(subset='isrc', keep='first')
deezer_sample = deezer.sample(n=100000, random_state=42069)
deezer_sample.head(10)

,title,artist,album,duration,deezer_link,rank,explicit_lyrics,release_date,id,isrc
22062,Blew My Cool Over You,Herman Brood & His Wild Romance,My Way - The Hits,187,https://www.deezer.com/track/1024549,92517,False,2001-10-04,1024549,NLA200100088
141001,You And I,Rick James,Funked!: Volume 3 1977-1980,480,https://www.deezer.com/track/1141814,140622,False,2009-03-02,1141814,USMO17800016
358150,"Somehow, someway",Organized Konfusion,The Best Of Organized Konfusion,184,https://www.deezer.com/track/1359524,6960,False,2008-06-17,1359524,USA560785624
494327,Siren 2,Sound Effects,Special FX2,20,https://www.deezer.com/track/1496615,1798,False,2005-02-22,1496615,USA560510544
207778,Just In Time - Original,Lee Morgan,Expoobident,216,https://www.deezer.com/track/1208718,26974,False,2006-03-28,1208718,USFB20600383
155181,Introduction by Symphony Sid,Mongo Santamaria,Mongo At The Village Gate,41,https://www.deezer.com/track/1156021,15272,False,2010-12-21,1156021,06UMGOT00435
392601,Señora Chichera,Liliana Herrero,Recuerdos De Provincia,170,https://www.deezer.com/track/1394070,25505,False,2003-01-01,1394070,ARF669900006
274055,Venus,Rolando Ojeda,Perlas De Amor,158,https://www.deezer.com/track/1275191,9,False,2008-08-04,1275191,ES5060849901
357404,Cute,Al Casey,"A Tribute To ""Fats""",236,https://www.deezer.com/track/1358778,2224,False,2006-06-13,1358778,DEB689400006
132037,Against All Odds (Take A Look At Me Now) (Albu...,Mariah Carey featuring Westlife,Unbreakable:The Greatest Hits,200,https://www.deezer.com/track/1132790,2708,False,1980-01-01,1132790,USSM10015027


In [ ]:
# get position of a song based on isrc if the scrpit below fails
if not deezer_sample[deezer_sample['isrc'] == 'USA560818342'].empty:
	position = deezer_sample.index.get_loc(deezer_sample.loc[deezer_sample['isrc'] == 'USA560818342'].iloc[0].name)
	print("Position in the sample:", position)
else:
	print("The specified ISRC 'US2Y30622939' does not exist in the sample.")

Position in the sample: 33935


Position in the sample: 61693


In [4]:

access_token = get_token()
with open('../../Data/Raw/spotify_deezer_data_1.txt', "a+", encoding="utf-8") as f:
    for index, row in deezer_sample.iloc[61693:75000].iterrows():
        this_song = (index, row['isrc'])
        if index % 1000 == 0:
            print("Index: ", index)
        # The token expires in around an hour, so we need to get a new one every so often
        if index % 5000 == 0:
            access_token = get_token()
        try:
            track_info = get_track_by_isrc(access_token, row['isrc'])
            # print(track_info.json()['tracks']['items'])
            f.write(str(track_info.json()['tracks']["items"][0]) + '\n')
        except Exception as e:
            print("Failed to get a track: ", this_song)
            try:
                print("With Python throwing: ", e)
                print("With the response message: ", track_info.json())
            except Exception as e:
                # We don't get the tempalted response from spotify, meaning we get a error message instead
                # Very likely just a 429 error, so we wait a bit and try again
                # Or in some cases other issues, leading to 529/other errors
                print(track_info)
                time.sleep(300)
            # access_token = get_token()

        time.sleep(0.1)

Failed to retrieve access token: 
<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>502 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>



UnboundLocalError: local variable 'access_token' referenced before assignment